In [4]:
import numpy as np
import tensorflow as tf

nb_entities = 100
nb_predicates = 1

entity_embedding_size = 10
predicate_embedding_size = 10


# Instantiating entity and predicate embedding layers
entity_embedding_layer = tf.get_variable('entities',
                                         shape=[nb_entities + 1, entity_embedding_size],
                                         initializer=tf.contrib.layers.xavier_initializer())

predicate_embedding_layer = tf.get_variable('predicates',
                                            shape=[nb_predicates + 1, predicate_embedding_size],
                                            initializer=tf.contrib.layers.xavier_initializer())

from inferbeddings.models import base as models
from inferbeddings.models import similarities

# Instantiating the model parameters
model_class = models.get_function('DistMult')
similarity_function = similarities.get_function('dot')

model_parameters = dict(similarity_function=similarity_function)

In [2]:
from inferbeddings.knowledgebase import Fact, KnowledgeBaseParser

# Knowledge Base triples
triples = [
    ('a', 'p', 'b'),
    ('c', 'p', 'd'),
    ('a', 'q', 'b')
]

def fact(s, p, o):
    return Fact(predicate_name=p, argument_names=[s, o])

facts = [fact(s, p, o) for s, p, o in triples]
parser = KnowledgeBaseParser(facts)

from inferbeddings.parse import parse_clause

# Clauses
clause_str = 'q(x, y) :- p(x, y)'
clauses = [parse_clause(clause_str)]

In [5]:
from inferbeddings.adversarial import Adversarial

# Adversary - used for computing the adversarial loss
adversarial = Adversarial(clauses=clauses,
                          parser=parser,
                          entity_embedding_layer=entity_embedding_layer,
                          predicate_embedding_layer=predicate_embedding_layer,
                          model_class=model_class,
                          model_parameters=model_parameters,
                          batch_size=1)

In [ ]:
violation_errors, violation_loss = adversarial.errors, adversarial.loss

violation_finding_optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)
violation_training_step = violation_finding_optimizer.minimize(- violation_loss, var_list=adversarial.parameters)

init_op = tf.global_variables_initializer()

with tf.Session() as session:
    session.run(init_op)
    
    